<a href="https://colab.research.google.com/github/JayThibs/ai-safety-scrape/blob/main/scrape_ai_alignment_content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting Contents from AI Alignment Resources

This notebook is used to scrape the contents of AI Alignment resources.

# Installations

In [1]:
!python --version

Python 3.7.12


In [2]:
%env PYTHONPATH=

env: PYTHONPATH=


In [ ]:
%%bash

MINICONDA_INSTALLER_SCRIPT=Miniconda3-py37_4.11.0-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [16]:
%%bash

conda install --channel defaults conda python=3.7 --yes
conda update --channel defaults --all --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - conda
    - python=3.7


The following packages will be UPDATED:

  ca-certificates    anaconda::ca-certificates-2020.10.14-0 --> pkgs/main::ca-certificates-2021.10.26-h06a4308_2
  certifi                anaconda::certifi-2020.6.20-py37_0 --> pkgs/main::certifi-2021.10.8-py37h06a4308_2


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pandoc-2.12                |       h06a4308_0         9.5 MB
    ------------------------------------------------------------
                                           Total:         9.5 MB

The following packa

In [17]:
import sys

_ = (sys.path.append("/usr/local/lib/python3.7/site-packages"))

In [18]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/usr/local/lib/python3.7.12/site-packages',
 '/usr/local/lib/python3.7/site-packages']

In [19]:
!conda install -c conda-forge pandoc --yes

Solving environment: | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pandoc


The following NEW packages will be INSTALLED:

  python_abi         conda-forge/linux-64::python_abi-3.7-2_cp37m

The following packages will be UPDATED:

  pandoc                  pkgs/main::pandoc-2.12-h06a4308_0 --> conda-forge::pandoc-2.17.1.1-ha770c72_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2021.10.26~ --> conda-forge::ca-certificates-2021.10.8-ha878542_0
  certifi            pkgs/main::certifi-2021.10.8-py37h06a~ --> conda-forge::certifi-2021.10.8-py37h89c1867_1
  conda              pkgs/main::conda-4.11.0-py37h06a4308_0 --> conda-forge::conda-4.11.0-py37h89c1867_0


Preparing transaction: \ done
Verifying transaction: / done
Executing transaction: \ done


In [20]:
!conda install -c anaconda pip --yes

Solving environment: - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pip


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2021.10.~ --> anaconda::ca-certificates-2020.10.14-0
  certifi            conda-forge::certifi-2021.10.8-py37h8~ --> anaconda::certifi-2020.6.20-py37_0
  conda              conda-forge::conda-4.11.0-py37h89c186~ --> pkgs/main::conda-4.11.0-py37h06a4308_0
  pip                  pkgs/main::pip-21.2.2-py37h06a4308_0 --> anaconda::pip-20.2.4-py37_0


Preparing transaction: | done
Verifying transaction: - \ done
Executing transaction: / - done


In [21]:
!pip install --upgrade pandoc -q

In [22]:
!pip install tika arxiv

# Imports

In [23]:
import os
import pandas as pd
from tika import parser
from google.colab import drive
from pathlib import Path
from urllib import request
from bs4 import BeautifulSoup
import arxiv
import pandoc
import requests
import tarfile
import pickle

# Setting up Environment

In [24]:
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [26]:
CODE_DIR = Path('.') / 'code-projects/gpt-ai-safety'
RAW_DIR = Path('.') / 'data/raw'
TARS_DIR = RAW_DIR / 'tars'
LATEX_DIR = RAW_DIR / 'latex_files'
PDFS_DIR = RAW_DIR / 'pdfs'

In [27]:
%cd {CODE_DIR}

/content/drive/MyDrive/code-projects/gpt-ai-safety


In [28]:
# !git clone https://github.com/JayThibs/ai-safety-scrape
# !mv ai-safety-scrape/* .

# Load and Explore Data

In [29]:
df = pd.read_csv('ai-alignment-papers.csv')
df.head()

,Key,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,DOI,Url,Abstract Note,Date,Date Added,Date Modified,Access Date,Pages,Num Pages,Issue,Volume,Number Of Volumes,Journal Abbreviation,Short Title,Series,Series Number,Series Text,Series Title,Publisher,Place,Language,Rights,Type,Archive,Archive Location,Library Catalog,Call Number,Extra,Notes,File Attachments,Link Attachments,Manual Tags,...,Cast Member,Commenter,Composer,Cosponsor,Counsel,Interviewer,Producer,Recipient,Reviewed Author,Scriptwriter,Words By,Guest,Number,Edition,Running Time,Scale,Medium,Artwork Size,Filing Date,Application Number,Assignee,Issuing Authority,Country,Meeting Name,Conference Name,Court,References,Reporter,Legal Status,Priority Numbers,Programming Language,Version,System,Code,Code Number,Section,Session,Committee,History,Legislative Body
0,XBZAPQFK,blogPost,2020.0,"Kokotajlo, Daniel",Three kinds of competitiveness,AI Impacts,NaN,NaN,NaN,https://aiimpacts.org/three-kinds-of-competiti...,"By Daniel Kokotajlo In this post, I distinguis...",2020-03-30,2022-01-30 01:53:10,2022-01-30 01:53:10,2021-11-20 18:55:39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en-US,NaN,NaN,NaN,NaN,NaN,NaN,ZSCC: NoCitationData[s0] ACC: N/A Section: Blog,NaN,/Users/jacquesthibodeau/Zotero/storage/PU9A2KS...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HX9UZ5JP,journalArticle,2020.0,"Cihon, Peter; Maas, Matthijs M.; Kemp, Luke",Fragmentation and the Future: Investigating Ar...,Global Policy,NaN,1758-5899,10.1111/1758-5899.12890,https://onlinelibrary.wiley.com/doi/abs/10.111...,The international governance of artificial int...,2020,2022-01-30 04:47:43,2022-01-30 04:47:43,2021-11-13 15:58:24,545-556,NaN,5,11.0,NaN,NaN,Fragmentation and the Future,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN,NaN,NaN,NaN,Wiley Online Library,NaN,ZSCC: 0000010 _eprint: https://onlinelibrary....,NaN,/Users/jacquesthibodeau/Zotero/storage/2TZBI3F...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BQCZM53S,blogPost,2021.0,"Clarke, Sam; Martin, Samuel Dylan",Distinguishing AI takeover scenarios,AI Alignment Forum,NaN,NaN,NaN,https://www.alignmentforum.org/posts/qYzqDtoQa...,Epistemic status: lots of this involves interp...,2021-09-08,2022-01-30 04:47:42,2022-01-30 04:47:42,2021-11-18 23:45:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSCC: NoCitationData[s0] ACC: N/A,NaN,/Users/jacquesthibodeau/Zotero/storage/ENAMQXC...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JVMJ4RMM,journalArticle,2020.0,"Stray, Jonathan",Aligning AI Optimization to Community Well-Being,International Journal of Community Well-Being,NaN,"2524-5295, 2524-5309",10.1007/s42413-020-00086-3,http://link.springer.com/10.1007/s42413-020-00...,NaN,2020-12,2022-01-30 04:47:36,2022-01-30 04:47:36,2021-11-13 22:47:54,443-463,NaN,4,3.0,NaN,Int. Journal of Com. WB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN,NaN,NaN,NaN,DOI.org (Crossref),NaN,ZSCC: 0000010,NaN,/Users/jacquesthibodeau/Zotero/storage/V3BEV7X...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,W8F6VI9I,thesis,2020.0,"Shah, Rohin Monish",Extracting and Using Preference Information fr...,NaN,NaN,NaN,NaN,https://www.proquest.com/openview/da8bf63ef343...,Typically when learning about what people want...,2020-12-17,2022-01-30 04:47:35,2022-01-30 04:47:35,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"University of California, Berkeley","Berkeley, CA",en,NaN,NaN,NaN,NaN,Zotero,NaN,ZSCC: 0000000,NaN,/

In [30]:
df['Item Type'].unique()

array(['blogPost', 'journalArticle', 'thesis', 'conferencePaper',
       'manuscript', 'report', 'bookSection', 'magazineArticle', 'book'],
      dtype=object)

In [31]:
item_nums = []
for item in df['Item Type'].unique():
    item_nums.append([item, len(df[df['Item Type'] == item])])

item_nums.sort(key=lambda x:x[1])
item_nums

[['magazineArticle', 2],
 ['thesis', 3],
 ['book', 13],
 ['bookSection', 52],
 ['report', 87],
 ['manuscript', 154],
 ['journalArticle', 170],
 ['conferencePaper', 262],
 ['blogPost', 421]]

In [32]:
df[df['Item Type'] == 'journalArticle'].head()

,Key,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,DOI,Url,Abstract Note,Date,Date Added,Date Modified,Access Date,Pages,Num Pages,Issue,Volume,Number Of Volumes,Journal Abbreviation,Short Title,Series,Series Number,Series Text,Series Title,Publisher,Place,Language,Rights,Type,Archive,Archive Location,Library Catalog,Call Number,Extra,Notes,File Attachments,Link Attachments,Manual Tags,...,Cast Member,Commenter,Composer,Cosponsor,Counsel,Interviewer,Producer,Recipient,Reviewed Author,Scriptwriter,Words By,Guest,Number,Edition,Running Time,Scale,Medium,Artwork Size,Filing Date,Application Number,Assignee,Issuing Authority,Country,Meeting Name,Conference Name,Court,References,Reporter,Legal Status,Priority Numbers,Programming Language,Version,System,Code,Code Number,Section,Session,Committee,History,Legislative Body
1,HX9UZ5JP,journalArticle,2020.0,"Cihon, Peter; Maas, Matthijs M.; Kemp, Luke",Fragmentation and the Future: Investigating Ar...,Global Policy,NaN,1758-5899,10.1111/1758-5899.12890,https://onlinelibrary.wiley.com/doi/abs/10.111...,The international governance of artificial int...,2020,2022-01-30 04:47:43,2022-01-30 04:47:43,2021-11-13 15:58:24,545-556,NaN,5,11.0,NaN,NaN,Fragmentation and the Future,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN,NaN,NaN,NaN,Wiley Online Library,NaN,ZSCC: 0000010 _eprint: https://onlinelibrary....,NaN,/Users/jacquesthibodeau/Zotero/storage/2TZBI3F...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JVMJ4RMM,journalArticle,2020.0,"Stray, Jonathan",Aligning AI Optimization to Community Well-Being,International Journal of Community Well-Being,NaN,"2524-5295, 2524-5309",10.1007/s42413-020-00086-3,http://link.springer.com/10.1007/s42413-020-00...,NaN,2020-12,2022-01-30 04:47:36,2022-01-30 04:47:36,2021-11-13 22:47:54,443-463,NaN,4,3.0,NaN,Int. Journal of Com. WB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN,NaN,NaN,NaN,DOI.org (Crossref),NaN,ZSCC: 0000010,NaN,/Users/jacquesthibodeau/Zotero/storage/V3BEV7X...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,TK5F29IU,journalArticle,2021.0,"Hayden, Benjamin; Niv, Yael",The case against economic values in the orbito...,Behavioral Neuroscience,NaN,NaN,10.1037/bne0000448,https://osf.io/7hgup,Much of traditional neuroeconomics proceeds fr...,2021,2022-01-30 04:48:47,2022-01-30 04:48:47,2021-11-08 23:41:47,192-201,NaN,2,135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DOI.org (Crossref),NaN,ZSCC: 0000026 DOI: 10.31234/osf.io/7hgup,NaN,NaN,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NHWZIKZ2,journalArticle,2020.0,"Fernandes, Pedro; Santos, Francisco C.; Lopes,...",Norms for Beneficial A.I.: A Computational Ana...,AI Communications,NaN,"18758452, 09217126",10.3233/AIC-201502,http://arxiv.org/abs/1907.03843,The rise of artificial intelligence (A.I.) bas...,2020-12-18,2022-01-30 04:48:46,2022-01-30 04:48:46,2021-11-13 22:40:37,155-171,NaN,3-6,33.0,NaN,AIC,Norms for Beneficial A.I.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,arXiv.org,NaN,ZSCC: 0000004 arXiv: 1907.03843,NaN,/Users/jacquesthibodeau/Zotero/storage/JAVXSVN...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,HDWGJGAP,journalArticle,2021.0,"Mingard, Chris; Valle-Pérez, Guillermo; Skalse...","Is SGD a Bayesian sampler? Well, almost",Journal of Machine Learning Research,NaN,NaN,NaN,http://arxiv.org/abs/2006.15191,Overparameterised deep neural networks (DNNs) ...,2021-02,2022-01-30 04:48:46,2022-01-30 04:48:46,2021-11-13 22:56:31,NaN,NaN,NaN,22.

# Extract text from AI Alignment Resources

In [33]:
df_arxiv = df[df['Url'].str.contains('arxiv') == True]
arxiv_paper_list = df_arxiv['Url'].values
arxiv_paper_list[0:10]

array(['http://arxiv.org/abs/2002.11328',
       'http://arxiv.org/abs/2010.11645',
       'http://arxiv.org/abs/2002.11708',
       'http://arxiv.org/abs/2012.10800',
       'http://arxiv.org/abs/2011.08512',
       'http://arxiv.org/abs/2002.11089',
       'http://arxiv.org/abs/2012.05876',
       'http://arxiv.org/abs/2010.14603',
       'http://arxiv.org/abs/2005.01643',
       'http://arxiv.org/abs/1907.03843'], dtype=object)

In [34]:
# # Grabbing files from ar5iv in case source (LaTeX doesn't work)

# def grab_text_from_webpage(url):
#     with request.urlopen(url) as response:
#         html = response.read()
#     soup = BeautifulSoup(html, "html.parser")
#     return soup.get_text(separator=" ")

# df_arxiv['Url'].str.replace('arxiv', 'ar5iv')
# arxiv_paper = grab_text_from_webpage('http://ar5iv.org/abs/2002.11328')
# arxiv_paper_list = arxiv_paper.split('\n\n')
# list_text = BeautifulSoup(arxiv_paper, "html.parser").get_text(separator=" ").split('\n')
# ''.join(list_text)

## Extracting PDFs with Tika

In [35]:
# def tikaTextExtractor(file_path):
#     """Extracts text from a PDF using tika."""
#     print("Extracting text from file: " + file_path)
#     parsed_tika = parser.from_file(file_path)
#     return parsed_tika["content"]

# txt = tikaTextExtractor(str(RAW_DIR / 'pdfs' / 'Superintelligence.pdf'))

In [36]:
# print(txt[0:1000])

In [37]:
# with open(RAW_DIR / 'txts' / 'Superintelligence.txt', 'w') as f:
#     f.write(txt)

In [38]:
# with request.urlopen('https://arxiv.org/e-print/2010.11645') as response:
#     html = response.read()

In [66]:
# for i,link in enumerate(['https://arxiv.org/e-print/2010.11645']):
#     r = requests.get(link,allow_redirects=True)
#     with open(f"a{i}.","wb") as code:
#         code.write(r.content)

## Extract LaTeX papers from arXiv

In [39]:
paper = next(arxiv.Search(id_list=["2101.00027v1"]).results())

In [40]:
# paper.download_source(dirpath=TARS_DIR, filename="2101.00027v1.tar.gz")

In [41]:
paper.entry_id

'http://arxiv.org/abs/2101.00027v1'

In [42]:
if not os.path.exists(TARS_DIR / "211.00027v1.tar.gz"):
    print('does not exists')

does not exists


In [43]:
def download_arxiv_paper_tar(paper_id, tar_filename):
    paper = next(arxiv.Search(id_list=[paper_id]).results())
    if not os.path.exists(str(TARS_DIR / tar_filename)):
        paper.download_source(dirpath=TARS_DIR, filename=tar_filename)

def extract_tex(tar_filename):
    tar = tarfile.open(str(TARS_DIR / tar_filename))
    for member in tar.getmembers():
        if ".tex" in member.name:
            tex_filename = member.name
            tar.extract(member, LATEX_DIR)
    tar.close()
    return tex_filename

In [44]:
pickled_arxiv_list = str(RAW_DIR / "arxiv_ids_and_names.pkl")

if not os.path.exists(pickled_arxiv_list):
    arxiv_ids_and_names = []
else:
    pickle.load(pickled_arxiv_list)

In [25]:
for paper_id in arxiv_paper_list:
    paper_id = paper_id.split('/')[-1] + "v1"
    tar_filename = paper_id + ".tar.gz"
    download_arxiv_paper_tar(paper_id, tar_filename)
    if not os.path.exists(str(LATEX_DIR / tar_filename)):
        tex_filename = extract_tex(tar_filename)
    arxiv_ids_and_names.append([paper_id, tex_filename])


NameError: ignored

In [ ]:
with open('arxiv_ids_and_names.pkl', 'wb') as b:
    pickle.dump(arxiv_ids_and_names, b)

## Convert LaTeX or EPUB with Pandoc

In [50]:
!timeout 10s pandoc -s the-pile.tex -o the-pile.md --wrap=none

Error at "the-pile.tex" (line 360, column 1):
unexpected }
expecting \end{document}
}
^


In [53]:


try:
    pandoc.read(file='the-pile.tex', format='latex')
except ExitCodeError:
    # import traceback
    # traceback.print_exc()
    pass

NameError: ignored

# Search all useful arxiv papers

In [32]:
search = arxiv.Search(
    query="ai safety",
    max_results=10,
    sort_by = arxiv.SortCriterion.Relevance
)

In [33]:
for result in search.results():
  print('Title: ', result.title, '\nDate: ',result.published , '\nId: ', result.entry_id, 
        '\nSummary: ',result.summary ,'\nURL: ', result.pdf_url, '\n\n')

Title:  Evolutionary Computation and AI Safety: Research Problems Impeding Routine and Safe Real-world Application of Evolution 
Date:  2019-06-24 19:26:10+00:00 
Id:  http://arxiv.org/abs/1906.10189v2 
Summary:  Recent developments in artificial intelligence and machine learning have
spurred interest in the growing field of AI safety, which studies how to
prevent human-harming accidents when deploying AI systems. This paper thus
explores the intersection of AI safety with evolutionary computation, to show
how safety issues arise in evolutionary computation and how understanding from
evolutionary computational and biological evolution can inform the broader
study of AI safety. 
URL:  http://arxiv.org/pdf/1906.10189v2 


Title:  AI Failures: A Review of Underlying Issues 
Date:  2020-07-18 15:31:29+00:00 
Id:  http://arxiv.org/abs/2008.04073v1 
Summary:  Instances of Artificial Intelligence (AI) systems failing to deliver
consistent, satisfactory performance are legion. We investigate w

# SOTAWHAT

In [24]:
!git clone https://github.com/chiphuyen/sotawhat
%cd sotawhat
!pip install .
%cd ..

Cloning into 'sotawhat'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 92 (delta 0), reused 1 (delta 0), pack-reused 89
Unpacking objects: 100% (92/92), done.
/content/drive/MyDrive/code-projects/gpt-ai-safety/sotawhat
Processing /content/drive/MyDrive/code-projects/gpt-ai-safety/sotawhat
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 2.7 MB 5.1 MB/s 
  Created wheel for sotawhat: filename=sotawhat-0.0.1-py3-none-any.whl size=4623 sha256=5aef8419ceec258d0ead1cf1725627fcb3cf7f5d4dfae58b5c359b5929819a17
  St

In [29]:
!sotawhat ai safety 10

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Impossibility Results in AI: A Survey (Mario Brcic - 1 September, 2021)
As such, these results serve as guidelines, reminders, and warnings to AI safety, AI policy, and governance researchers. We concluded that deductive impossibilities deny 100%-guarantees for security
Link: sp;[<a href=
Towards a framework for evaluating the safety, acceptability and efficacy of AI systems for health: an initial synthesis (Jessica Morley - 14 April, 2021)
There are numerous reasons for this, but a main contributory factor is the lack of internationally accepted, or formalised, regulatory standards to assess AI safety and impact and effectiveness. We do this by conducting a systematic search across Scopus, PubMed and Google Scholar to identify all the relevant literature published between January 1970 and November 2020 related to the evaluation of: output performance; efficacy; and real-world use 